# Run notebooks

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from glob import glob

import papermill as pm

import util

/glade/work/kristenk/miniconda/envs/cesm-exp/lib/python3.7/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


## Get control structure

In [3]:
control = util.get_control_dict()
control

{'data_sources': {'casename': 'g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.4p2z.001'},
 'output_root': './_computed-notebooks',
 'project': 'NCGD0011',
 'book_toc': {'format': 'jb-book',
  'root': 'description',
  'parts': [{'caption': 'Global overview',
    'chapters': [{'file': 'ocn-global-bgc-metrics'}]},
   {'caption': 'Macronutrients', 'chapters': [{'file': 'ocn-macronuts'}]},
   {'caption': 'Ocean Carbon Cycle',
    'chapters': [{'file': 'ocn-carbonfluxes'}]},
   {'caption': 'Phytoplankton',
    'chapters': [{'file': 'ocn-phyto-biomass'},
     {'file': 'ocn-phyto-prod'},
     {'file': 'ocn-phyto-lims'},
     {'file': 'ocn-chl'}]},
   {'caption': 'Zooplankton', 'chapters': [{'file': 'ocn-zoo'}]},
   {'caption': 'Dissolved iron', 'chapters': [{'file': 'ocn-iron'}]},
   {'caption': 'Ocean Tracers',
    'chapters': [{'file': 'intro-ocn-tracer-views',
      'sections': [{'glob': 'ocn-tracer-views*'}]}]},
   {'caption': 'Paper Figures', 'chapters': [{'file': 'ocn-paperfigs'}]}]},
 'book_c

In [4]:
util.setup_book()

## Spin up a cluster

The notebooks are configured to connect to this cluster.

In [5]:
cluster = util.get_Cluster(project=control['project'])
cluster.scale(32)
cluster

## Run all the notebooks

In [6]:
output_dir = control['output_dir']

for nb, info in control['compute_notebooks'].items():
    
    parameter_groups = info['parameter_groups']
    use_cluster = info['use_cluster']
    
    default_params = {}
    if 'default_params' in info:
        default_params = info['default_params']

    for key, parms in parameter_groups.items():

        input_path = f'{nb}.ipynb'
        output_path = (
            f'{output_dir}/{nb}-{key}.ipynb'
            if key != 'none' else f'{output_dir}/{nb}.ipynb'
        )

        # check notebook expectations
        nb_api = pm.inspect_notebook(input_path)
        
        # TODO: validate paramter and API

        if nb_api:
            parms_in = dict(**default_params)
            parms_in.update(dict(**parms))
            parms_in['cluster_scheduler_address'] = cluster.scheduler_address
        else:
            parms_in = {}

        print(f'executing {input_path}')
        o = pm.execute_notebook(
            input_path=input_path,
            output_path=output_path,
            kernel_name=info['kernel_name'],
            parameters=parms_in,
            engine_name='md_jinja',
            jinja_data=parms,
        )

executing description.ipynb


Executing:   0%|          | 0/2 [00:00<?, ?cell/s]

executing ocn-carbonfluxes.ipynb


Executing:   0%|          | 0/24 [00:00<?, ?cell/s]

executing ocn-global-bgc-metrics.ipynb


...

Executing:   0%|          | 0/34 [00:00<?, ?cell/s]

executing ocn-iron.ipynb


Executing:   0%|          | 0/29 [00:00<?, ?cell/s]

executing ocn-paperfigs.ipynb


Executing:   0%|          | 0/38 [00:00<?, ?cell/s]

In [7]:
cluster.close()